<a href="https://colab.research.google.com/github/waytogoasis/app/blob/main/fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U scikit-fuzzy

# 1. Acesso à Planilha e Autenticação
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Abra a planilha pelo NOME
spreadsheet_name = 'base'
try:
    spreadsheet = gc.open(spreadsheet_name)
    worksheet = spreadsheet.sheet1  # Acessa a primeira aba da planilha
except gspread.exceptions.SpreadsheetNotFound:
    print(f"Planilha '{spreadsheet_name}' não encontrada. Verifique o nome e as permissões de compartilhamento.")
    exit()

# 2. Leitura dos Dados da Planilha (Mantido)

data = worksheet.get_all_values()
header = data[0]
rows = data[1:]

# Cria um dicionário para armazenar os dados de cada turbina
turbines_data = {}
for row in rows:
    segment_turbine = row[0]
    segment = int(segment_turbine[0])
    turbine = segment_turbine[1]
    if turbine not in turbines_data:
        turbines_data[turbine] = {}
    if segment not in turbines_data[turbine]:
        turbines_data[turbine][segment] = {}

    turbines_data[turbine][segment] = {
        'intensities': [int(val) for val in row[1:]]
    }

# 3. Implementação da Lógica Fuzzy

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import random

def fuzzy_control(velocidade_vento, segmento):
    """
    Controla a intensidade do campo magnético usando lógica fuzzy.

    Args:
        velocidade_vento (float): Velocidade do vento no segmento (simulado).
        segmento (int): Número do segmento da turbina (1 a 5).

    Returns:
        float: Intensidade do campo magnético (0 a 10).
    """

    # Antecedent (Entrada)
    velocidade = ctrl.Antecedent(np.arange(0, 11, 1), 'velocidade')
    segmento_ctrl = ctrl.Antecedent(np.arange(1, 6, 1), 'segmento')

    # Consequent (Saída)
    intensidade = ctrl.Consequent(np.arange(0, 11, 1), 'intensidade')

    # Funções de pertinência para velocidade
    velocidade['baixa'] = fuzz.trimf(velocidade.universe, [0, 0, 5])
    velocidade['media'] = fuzz.trimf(velocidade.universe, [0, 5, 10])
    velocidade['alta'] = fuzz.trimf(velocidade.universe, [5, 10, 10])

    # Funções de pertinência para segmento
    segmento_ctrl['inicio'] = fuzz.trimf(segmento_ctrl.universe, [1, 1, 3])
    segmento_ctrl['meio'] = fuzz.trimf(segmento_ctrl.universe, [1, 3, 5])
    segmento_ctrl['fim'] = fuzz.trimf(segmento_ctrl.universe, [3, 5, 5])

    # Funções de pertinência para intensidade
    intensidade['fraca'] = fuzz.trimf(intensidade.universe, [0, 0, 5])
    intensidade['media'] = fuzz.trimf(intensidade.universe, [0, 5, 10])
    intensidade['forte'] = fuzz.trimf(intensidade.universe, [5, 10, 10])

    # Regras Fuzzy
    rule1 = ctrl.Rule(velocidade['baixa'] & segmento_ctrl['inicio'], intensidade['fraca'])
    rule2 = ctrl.Rule(velocidade['media'] & segmento_ctrl['meio'], intensidade['media'])
    rule3 = ctrl.Rule(velocidade['alta'] & segmento_ctrl['fim'], intensidade['forte'])
    rule4 = ctrl.Rule(velocidade['alta'] & segmento_ctrl['inicio'], intensidade['media'])
    rule5 = ctrl.Rule(velocidade['baixa'] & segmento_ctrl['fim'], intensidade['fraca'])
    rule6 = ctrl.Rule(velocidade['media'] & segmento_ctrl['inicio'], intensidade['media'])
    rule7 = ctrl.Rule(velocidade['baixa'] & segmento_ctrl['meio'], intensidade['fraca'])
    rule8 = ctrl.Rule(velocidade['media'] & segmento_ctrl['fim'], intensidade['media'])
    rule9 = ctrl.Rule(velocidade['alta'] & segmento_ctrl['meio'], intensidade['forte'])

    # Sistema de Controle Fuzzy
    intensidade_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
    intensidade_sim = ctrl.ControlSystemSimulation(intensidade_ctrl)

    # Simulação
    intensidade_sim.input['velocidade'] = velocidade_vento
    intensidade_sim.input['segmento'] = segmento
    intensidade_sim.compute()

    return intensidade_sim.output['intensidade']

# 4. Execução do Controle Fuzzy para cada Turbina

optimized_turbines = {}
for turbine, segments in turbines_data.items():
    print(f"Controlando a turbina {turbine}:")
    optimized_intensities = {}
    for segment, data in segments.items():
        # Simula uma velocidade do vento aleatória para cada segmento
        velocidade_vento = random.uniform(0, 10)
        print(f"  Segmento {segment}: Velocidade do Vento = {velocidade_vento:.2f}")

        # Obtém a intensidade otimizada usando lógica fuzzy
        intensidade = fuzzy_control(velocidade_vento, segment)
        optimized_intensities[segment] = [intensidade] * 5

    optimized_turbines[turbine] = optimized_intensities

# 5. Geração de Gráficos com Estilo e Salvamento na Pasta "fuzzy"

import matplotlib.pyplot as plt
from IPython.display import Image, display
from google.colab import drive
import os

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho para a pasta "fuzzy" dentro da pasta compartilhada
fuzzy_graficos_path = '/content/drive/MyDrive/1aoPq66KDLN75_5z7vyRbWCWazogtjevV/fuzzy'

# Criar a pasta "fuzzy" se ela não existir
if not os.path.exists(fuzzy_graficos_path):
    os.makedirs(fuzzy_graficos_path)

# Definir o estilo dos gráficos
plt.style.use('dark_background')
neon_colors = ['#00FF00', '#00FFFF', '#FF9900', '#FF00FF', '#FFFF00']

def generate_visualizations_fuzzy(optimized_turbines, original_data):
    """
    Gera visualizações gráficas dos resultados do Controle Fuzzy e as salva na pasta "fuzzy".

    Args:
        optimized_turbines (dict): Dicionário com os resultados do controle fuzzy.
        original_data (list): Dados originais da planilha.
    """

    original_intensities = {}
    for row in original_data[1:]:
        segment_turbine, *intensities = row
        turbine = segment_turbine[1]
        segment = int(segment_turbine[0])
        if turbine not in original_intensities:
            original_intensities[turbine] = {}
        original_intensities[turbine][segment] = [int(val) for val in intensities]

    # Gráfico 1
    plt.figure(figsize=(12, 6))
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        original_avg = [np.mean(original_intensities[turbine][seg]) for seg in sorted(original_intensities[turbine].keys())]
        optimized_avg = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        plt.plot(original_avg, label=f'Original - Turbina {turbine}', color=neon_colors[i%5], linestyle='--')
        plt.plot(optimized_avg, label=f'Fuzzy - Turbina {turbine}', color=neon_colors[i%5])
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Comparação das Intensidades Médias por Turbina')
    plt.legend()
    plt.savefig(f'{fuzzy_graficos_path}/grafico_1.png')
    plt.close()
    display(Image(filename=f'{fuzzy_graficos_path}/grafico_1.png'))

    # Gráfico 2
    plt.figure(figsize=(10, 5))
    segments = sorted(optimized_turbines['a'].keys())
    intensities = [optimized_turbines['a'][seg][0] for seg in segments]
    plt.bar(segments, intensities, color=neon_colors[0])
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade')
    plt.title('Intensidades por Segmento (Turbina a) - Controle Fuzzy')
    plt.savefig(f'{fuzzy_graficos_path}/grafico_2.png')
    plt.close()
    display(Image(filename=f'{fuzzy_graficos_path}/grafico_2.png'))

    # Gráfico 3
    plt.figure(figsize=(10, 5))
    original_b = [np.mean(original_intensities['b'][seg]) for seg in sorted(original_intensities['b'].keys())]
    optimized_b = [np.mean(optimized_turbines['b'][seg]) for seg in sorted(optimized_turbines['b'].keys())]
    plt.plot(original_b, label='Original', color=neon_colors[1], linestyle='--')
    plt.plot(optimized_b, label='Fuzzy', color=neon_colors[1])
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Comparação das Intensidades por Segmento (Turbina b)')
    plt.legend()
    plt.savefig(f'{fuzzy_graficos_path}/grafico_3.png')
    plt.close()
    display(Image(filename=f'{fuzzy_graficos_path}/grafico_3.png'))

    # Gráfico 4
    plt.figure(figsize=(8, 6))
    all_intensities = [[np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())] for turbine in sorted(optimized_turbines.keys())]
    plt.imshow(all_intensities, cmap='magma', aspect='auto')
    plt.colorbar(label='Intensidade Média')
    plt.yticks(np.arange(len(optimized_turbines.keys())), labels=sorted(optimized_turbines.keys()))
    plt.xticks(np.arange(5), labels=['1', '2', '3', '4', '5'])
    plt.xlabel('Segmento')
    plt.ylabel('Turbina')
    plt.title('Mapa de Calor das Intensidades - Controle Fuzzy')
    plt.savefig(f'{fuzzy_graficos_path}/grafico_4.png')
    plt.close()
    display(Image(filename=f'{fuzzy_graficos_path}/grafico_4.png'))

    # Gráfico 5
    fig, axs = plt.subplots(2, 3, figsize=(18, 10), sharey=True)
    fig.suptitle('Dispersão entre Intensidade e Segmento para cada Turbina')
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        row, col = i // 3, i % 3
        intensities = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        axs[row, col].scatter(range(1, 6), intensities, color=neon_colors[i%5])
        axs[row, col].set_title(f'Turbina {turbine}')
        axs[row, col].set_xlabel('Segmento')
        axs[row, col].set_ylabel('Intensidade Média')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f'{fuzzy_graficos_path}/grafico_5.png')
    plt.close()
    display(Image(filename=f'{fuzzy_graficos_path}/grafico_5.png'))

    # Gráfico 6
    plt.figure(figsize=(8, 5))
    intensities_e = [optimized_turbines['e'][seg][0] for seg in sorted(optimized_turbines['e'].keys())]
    plt.hist(intensities_e, bins=10, color=neon_colors[4], edgecolor='black')
    plt.xlabel('Intensidade')
    plt.ylabel('Frequência')
    plt.title('Histograma das Intensidades para a Turbina e - Controle Fuzzy')
    plt.savefig(f'{fuzzy_graficos_path}/grafico_6.png')
    plt.close()
    display(Image(filename=f'{fuzzy_graficos_path}/grafico_6.png'))

    # Gráfico 7
    plt.figure(figsize=(10, 6))
    plt.boxplot(list(map(list, zip(*all_intensities))), labels=['1', '2', '3', '4', '5'], patch_artist=True,
                boxprops=dict(facecolor=neon_colors[2], color='white'),
                capprops=dict(color='white'),
                whiskerprops=dict(color='white'),
                flierprops=dict(color='white', markeredgecolor='white'),
                medianprops=dict(color='white'))
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade')
    plt.title('Boxplot das Intensidades por Segmento - Controle Fuzzy')
    plt.savefig(f'{fuzzy_graficos_path}/grafico_7.png')
    plt.close()
    display(Image(filename=f'{fuzzy_graficos_path}/grafico_7.png'))

    # Gráfico 8
    plt.figure(figsize=(12, 6))
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        intensities = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        plt.plot(range(1, 6), intensities, marker='o', color=neon_colors[i%5], label=f'Turbina {turbine}')
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Intensidades por Segmento para cada Turbina - Controle Fuzzy')
    plt.xticks(range(1, 6))
    plt.legend()
    plt.savefig(f'{fuzzy_graficos_path}/grafico_8.png')
    plt.close()
    display(Image(filename=f'{fuzzy_graficos_path}/grafico_8.png'))

    # Gráfico 9
    num_vars = 5
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        intensities = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        intensities += intensities[:1]
        ax.plot(angles, intensities, linewidth=1, linestyle='solid', color=neon_colors[i%5], label=f"Turbina {turbine}")
        ax.fill(angles, intensities, color=neon_colors[i%5], alpha=0.25)
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    plt.xticks(angles[:-1], ['Segmento 1', '2', '3', '4', '5'])
    ax.set_rlabel_position(0)
    plt.yticks([2, 5, 8], ["2", "5", "8"], color="grey", size=7)
    plt.ylim(0, 10)
    plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 35.7 MB/s eta 0:00:00
Controlando a turbina a:
  Segmento 1: Velocidade do Vento = 6.01
  Segmento 2: Velocidade do Vento = 2.61
  Segmento 3: Velocidade do Vento = 5.69
  Segmento 4: Velocidade do Vento = 5.65
  Segmento 5: Velocidade do Vento = 3.51
Controlando a turbina b:
  Segmento 1: Velocidade do Vento = 4.72
  Segmento 2: Velocidade do Vento = 1.66
  Segmento 3: Velocidade do Vento = 4.87
  Segmento 4: Velocidade do Vento = 6.94
  Segmento 5: Velocidade do Vento = 6.22
Controlando a turbina c:
  Segmento 1: Velocidade do Vento = 8.32
  Segmento 2: Velocidade do Vento = 7.19
  Segmento 3: Velocidade do Vento = 0.10
  Segmento 4: Velocidade do Vento = 4.21
  Segmento 5: Velocidade do Vento = 7.14
Controlando a turbina d:
  Segmento 1: Velocidade do Vento = 8.45
  Segmento 2: Velocidade do Vento = 9.70
  Segmento 3: Velocidade do Vento = 3.18
  Segmento 4: Velocidade do Vento = 8.82
  Segmento 5: Velocidade do Vento = 0.73